# 🧩 Reto 3 – Oxígeno disuelto
**Objetivo:** evaluar riesgo ecológico según oxígeno disuelto.

**Datos:** `../data/muestras_calidad_agua.csv`

### Pasos
1. Carga el CSV en `df`.
2. Extrae `sitio` y `oxigeno`.
3. Crea `nivel_oxigeno` (bajo/medio/alto).
4. Crea `riesgo_biologico` (⚠ si oxígeno < 5).
5. Filtra y exporta `resultado_oxigeno.csv`.

> **Reto adicional:** porcentaje de sitios con riesgo.

In [9]:

# Celda 1: imports y carga del CSV
import pandas as pd
from pathlib import Path
from IPython.display import display

candidates = [Path('muestras_calidad_agua.csv'), Path('../data/muestras_calidad_agua.csv')]
path = next((p for p in candidates if p.exists()), None)
if path is None:
    raise FileNotFoundError("No se encontró 'muestras_calidad_agua.csv' en 'notebooks' ni en '../data'.")

df = pd.read_csv(path)
print("Archivo cargado:", path.resolve())
display(df.head())

Archivo cargado: C:\Users\Daniel Bernal\Downloads\Github\4-condicionales-grupo_4\notebooks\muestras_calidad_agua.csv


,sitio,pH,oxigeno,temperatura
0,Río Subachoque,7.2,7.8,18.5
1,Quebrada Honda (Facatativá),6.3,6.2,19.4
2,Río Botello (Madrid),6.8,5.9,22.7
3,Afluente San José (El Rosal),7.9,8.3,20.1
4,Río Villeta,8.7,6.1,28.3


In [10]:
# Celda 2: extraer 'sitio' y columna de oxígeno (adapta nombres)
if 'sitio' not in df.columns:
    raise KeyError("La columna 'sitio' no existe en el CSV.")

ox_possible = ['oxigeno', 'oxígeno', 'O2', 'o2']
ox_col = next((c for c in ox_possible if c in df.columns), None)
if ox_col is None:
    raise KeyError("No se encontró columna de oxígeno ('oxigeno', 'oxígeno', 'O2', ...).")

df = df[['sitio', ox_col]].rename(columns={ox_col: 'oxigeno'})
# asegurar que oxígeno sea numérico (NaN si no convertible)
df['oxigeno'] = pd.to_numeric(df['oxigeno'], errors='coerce')

print("Columnas actuales:", df.columns.tolist())
display(df.head())

Columnas actuales: ['sitio', 'oxigeno']


,sitio,oxigeno
0,Río Subachoque,7.8
1,Quebrada Honda (Facatativá),6.2
2,Río Botello (Madrid),5.9
3,Afluente San José (El Rosal),8.3
4,Río Villeta,6.1


In [11]:
# Celda 3: crear nivel_oxigeno y riesgo_biologico
import numpy as np

# Definición: bajo <5, medio 5-8 (incluye 5 y 8), alto >8
df['nivel_oxigeno'] = np.where(df['oxigeno'] < 5, 'bajo',
                       np.where((df['oxigeno'] >= 5) & (df['oxigeno'] <= 8), 'medio',
                                np.where(df['oxigeno'] > 8, 'alto', np.nan)))

# Riesgo biológico: '⚠' si oxígeno < 5
df['riesgo_biologico'] = np.where(df['oxigeno'] < 5, '⚠', '')

# Comprobaciones
print("Conteo por nivel de oxígeno:")
print(df['nivel_oxigeno'].value_counts(dropna=False))
display(df.head(20))

Conteo por nivel de oxígeno:
nivel_oxigeno
medio    7
bajo     2
alto     1
Name: count, dtype: int64


,sitio,oxigeno,nivel_oxigeno,riesgo_biologico
0,Río Subachoque,7.8,medio,
1,Quebrada Honda (Facatativá),6.2,medio,
2,Río Botello (Madrid),5.9,medio,
3,Afluente San José (El Rosal),8.3,alto,
4,Río Villeta,6.1,medio,
5,Quebrada Cune (San Francisco),4.5,bajo,⚠
6,Río Tobia (Nocaima),5.2,medio,
7,Quebrada Negra (La Vega),7.0,medio,
8,Río Bahamón (Sasaima),6.7,medio,
9,Río Guayuriba (La Mesa),4.8,bajo,⚠


In [12]:
# Celda 4: exportar y calcular porcentaje de sitios con riesgo
resultado = df[['sitio', 'oxigeno', 'nivel_oxigeno', 'riesgo_biologico']].copy()
out_path = Path('resultado_oxigeno.csv')
resultado.to_csv(out_path, index=False)
print(f"Archivo guardado en: {out_path.resolve()}  (sobrescribe si existe)")

# Reto adicional: porcentaje de sitios con riesgo (si algún registro del sitio tiene ⚠)
sites_with_risk = resultado.groupby('sitio')['riesgo_biologico'].apply(lambda s: (s == '⚠').any())
pct_riesgo = 100 * sites_with_risk.sum() / sites_with_risk.size if sites_with_risk.size else 0.0
print(f"Registros procesados: {len(resultado)}")
print(f"Porcentaje de sitios con riesgo biológico: {pct_riesgo:.1f}%")

# Mostrar resultados por sitio con riesgo
print("\nSitios con riesgo (True = tiene al menos un registro con ⚠):")
display(sites_with_risk.reset_index(name='tiene_riesgo').sort_values('tiene_riesgo', ascending=False))


Archivo guardado en: C:\Users\Daniel Bernal\Downloads\Github\4-condicionales-grupo_4\notebooks\resultado_oxigeno.csv  (sobrescribe si existe)
Registros procesados: 10
Porcentaje de sitios con riesgo biológico: 20.0%

Sitios con riesgo (True = tiene al menos un registro con ⚠):


,sitio,tiene_riesgo
1,Quebrada Cune (San Francisco),True
6,Río Guayuriba (La Mesa),True
0,Afluente San José (El Rosal),False
2,Quebrada Honda (Facatativá),False
3,Quebrada Negra (La Vega),False
4,Río Bahamón (Sasaima),False
5,Río Botello (Madrid),False
7,Río Subachoque,False
8,Río Tobia (Nocaima),False
9,Río Villeta,False
